# Homework 1 - Streaming (15 pts)

### DUE: 02/18/2020 at 5:30pm

This homework is for practicing Python’s *generators* and the design of *streaming* algorithms in general. We’re going to use the **taxi_events.csv** and **taxi_trips.csv** data sets. Both of them are available on the class resources under *Data Sets* section. You are required to turn in this notebook with all the parts filled in place of <###>. Your notebook must be named BDM\_HW1\_LastName.ipynb.

## Task 1 (8 points)

Your task is to **compute the maximum number of active taxi cabs** that were hired at any point in time. This the same as computing the maximum number of taxi cabs that have passengers. The input data set is **taxi_events.csv**, which logged all pick-up and drop-off events for all taxi trips. The description of the fields in this file is as follows:

|Column name|Description|
|--|--|
|time |The timestamp of the event. All events are sorted increasingly by their timestamps. |
|vehicle_id |The unique ID of the taxi vehicle involved in this event. |
|event |A string of either *"pickup"* or *"dropoff"* for describing the drop-off or pick-up event, respectively. |

For example, let's assume that on *Feb-01-2015*, there was a taxi that picked up a passenger at midnight and dropped her off at 14 minutes past midnight. If the taxi cab has the Vehicle ID of *V102*, then you should see two events being logged in this data set as:

<pre>
...
2015-02-01 00:00:00+00,V102,pickup
...
2015-02-01 00:14:00+00,V102,dropoff
...
</pre>

You are given the above data set in a streaming fashion (reading in row by row), and must design a streaming algorithm that uses the least possible additional memory to compute the maximum number of hired taxi cabs at any point in time. Again, this is equivalent to having a virtual dispatcher, who repeatedly ask every second *"how many taxis are being hired (having passengers) at the moment?"*, and then log the maximum number during the entire period.

Please modify the code snippet below to complete this task. Your code should only output a single number, which is the maximum number of hired taxi cabs. Of course, you can add additional initialization codes outside of the for loop as needed. Additional, please provide a brief rationale and/or justification for your design after the code.

In [163]:
import csv
import datetime
from datetime import datetime as dt
import time

hr=0
def csvRows(filename):
    with open(filename, 'r') as fi:
        reader = csv.DictReader(fi)
        for row in reader:
            yield row

maxHiredCabs = {}

iterable = csvRows('taxi_events.csv')
threshold = datetime.timedelta(minutes=60)
delta = threshold
for row in iterable:
    t = time.strptime(row['time'],'%Y-%m-%d %H:%M:%S+%f')
    yy = t.tm_year
    mm = t.tm_mon
    dd = t.tm_mday
    H = t.tm_hour
    M = t.tm_min
    S = t.tm_sec
    current_t = dt(yy,mm,dd,H,M,S)
    diff = current_t-delta 
    #datetime = datetime - timedelta
    #datetime - datetime = timedelta
    if len(maxHiredCabs)==0: 
        duration =  current_t
    else:
        
        
    maxHiredCabs[duration]=maxHiredCabs.get(duration,0)+1
    
large = max(maxHiredCabs.values())
keyTime = list(maxHiredCabs.keys())[list(maxHiredCabs.values()).index(large)]

#print(max(maxHiredCabs.values()))
#print(list(maxHiredCabs.keys())[list(maxHiredCabs.values()).index(large)])

'''The max value without accounting for pickup/ dropoff is 57'''
'''Count unique vehicle_ids'''

uniqueHiredCabs = {}
for event in csvRows('taxi_events.csv'):
    time = event['time']
    if time==keyTime:
        vehicle = event['vehicle_id']
        uniqueHiredCabs[vehicle]=uniqueHiredCabs.get(vehicle,0)+1

#print(uniqueHiredCabs)

count = len(uniqueHiredCabs)

print(large)

{datetime.datetime(2015, 1, 31, 23, 0): 92400}


#### RATIONALE AND JUSTIFICATION

*We create a dictionary with time as the unique keys and a running count for how many events are called during that hour. We then take the max value.
For robustness, we consider the case where a single cab can do multiple pickups and dropoffs within an hour, so we look for unique vehicle ids within this hour*

## Task 2 (7 points)

The objective of this task is identical to Task 1's but you are asked to use the full **taxi_trips.csv** data set instead of the events. The main difference (and challenge) is that both pick-up and drop-off event for each trip is now presented in a single record, thus, the drop-off events are not sorted by their timestamps. You are again asked to do this in a streaming fashion that needs to minimize the amount of memory usage. Please modify the code below accordingly, and also with a brief explaination of the solution.

Below is the description of the **taxi_trips.csv** file, which is sorted only by the pick-up time:

|Column name|Description|
|--|--|
|trip_duration |The duration of the trip in seconds. This field is for your convenience since it can be derived also from the pick-up and drop-off times. |
|pickup_time |The timestamp of the pick-up of the trip. All trip records are sorted increasingly by their pick-up times. |
|dropoff_time |The timestamp of the drop-off event. |
|vehicle_id |The unique ID of the taxi vehicle involved in this trip record. |



In [64]:
import csv

def csvRows(filename):
    with open(filename, 'r') as fi:
        reader = csv.DictReader(fi)
        for row in reader:
            yield row

maxHiredCabs = {}
for row in csvRows('taxi_trips.csv'):
    pickup = row['pickup_time']
    dropoff = row['dropoff_time']
    vehicle = row['vehicle_id']
    #print (row)
    duration = row['trip_duration']
    if int(duration) > 60:
        maxHiredCabs[pickup]=maxHiredCabs.get(pickup,0)+1
        maxHiredCabs[dropoff]=maxHiredCabs.get(dropoff,0)+1
        
    
large = max(maxHiredCabs.values())
keyTime = list(maxHiredCabs.keys())[list(maxHiredCabs.values()).index(large)]
    
#print(maxHiredCabs)
print(large)

57


#### RATIONALE AND JUSTIFICATION

*In this case, we have to make sure we can add a cab regardless of its pickup or dropoff time. They are separate events.  The if statement is for glitches within 1 minute as that is the chosen time increment from Problem 1: namely, when a ride gets cancelled but picks someone else up right away.  This avoids double counting.*

In [56]:
#from datetime import datetime as dt
import time
import datetime
from datetime import datetime as dt, timedelta
s='2015-02-05 08:52:00+00'
p = '2015-02-05 08:52:00'

In [42]:
t= time.strptime(s,'%Y-%m-%d %H:%M:%S+%f')
t_prime =time.strptime(p,'%Y-%m-%d %H:%M:%S')
t

time.struct_time(tm_year=2015, tm_mon=2, tm_mday=5, tm_hour=8, tm_min=52, tm_sec=0, tm_wday=3, tm_yday=36, tm_isdst=-1)

In [111]:
dt_comp = (t.tm_year,t.tm_mon,t.tm_mday,t.tm_hour,t.tm_min,t.tm_sec)

In [114]:
a=dt(t.tm_year,t.tm_mon,t.tm_mday,t.tm_hour,t.tm_min,t.tm_sec)

In [116]:
dt(t.tm_year,t.tm_mon,t.tm_mday,t.tm_hour,t.tm_min,t.tm_sec)-dt(t.tm_year,t.tm_mon,t.tm_mday,t.tm_hour,t.tm_min,t.tm_sec)

datetime.timedelta(0)

In [155]:
b = datetime.datetime(2015,8,25,0,0,0,0)
a = datetime.datetime(2015,8,25,0,0,0,0)
c=a-datetime.timedelta(minutes=60)
d= a-b
d<datetime.timedelta(minutes=60)
type(d),type(c)

(datetime.timedelta, datetime.datetime)

In [123]:
datetime.timedelta(minutes=61)


datetime.timedelta(seconds=3660)

In [124]:
datetime.datetime?

In [135]:
maxHiredCabs = {}
maxHiredCabs[0]=maxHiredCabs.get(0,0)+1
maxHiredCabs 

{0: 1}

In [140]:
timedelta?

In [149]:
c.min

datetime.datetime(1, 1, 1, 0, 0)

ValueError: year 0 is out of range